In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import os
import math

In [2]:
DATA_BASE_PATH = 'DATA'
GENERATOR = '1'
RESULTS_BASE_PATH = 'RESULTS'

NODES = [(5, 8), (8, 5), (5, 12), (8, 12)]

# FAILUER_TIME = 16000

In [21]:
def load_sequence(file):
    Y = np.load(os.path.join(DATA_BASE_PATH, GENERATOR, f"{file}.npy"))
    Y = Y.reshape((-1))

    return Y

def split_sequence(sequence, n_steps_in, n_steps_out):
    X = [sequence[i:i+n_steps_in]
         for i in range(len(sequence) - (n_steps_in + n_steps_out) + 1)]
    y = [sequence[i:i+n_steps_out]
         for i in range(n_steps_in, len(sequence) - n_steps_out + 1)]

    return np.array(X).reshape(len(X), n_steps_in), np.array(y)

def define_model(n_steps_in, n_steps_out) -> Sequential:
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    
    return model

def run_experiment(exp, file, n_steps_in, n_steps_out, train_size, epochs=5, chunk_size=0):
    sequence = load_sequence(file)
    X, y = split_sequence(sequence, n_steps_in, n_steps_out)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42, shuffle=False)
    model = define_model(n_steps_in, n_steps_out)
    model.fit(X_train, y_train, epochs=epochs)

    if(chunk_size):
        
        chunks_number = math.floor(X_test.shape[0] / chunk_size)
        max_samples = chunks_number * chunk_size
        y_test = y_test[:max_samples]
        X_test = X_test[:max_samples]

        y_predicted = np.zeros(y_test.shape)

        X_test_chunks = np.array(np.array_split(X_test, chunks_number))
        y_test_chunks = np.array(np.array_split(y_test, chunks_number))

        for i in range(chunks_number):
            y_predicted[i * chunk_size : (i + 1) * chunk_size] = model.predict(X_test_chunks[i])
            model.fit(X_test_chunks[i], y_test_chunks[i], epochs)

    else:
        y_predicted = model.predict(X_test)

    exp_path = os.path.join(RESULTS_BASE_PATH, exp)

    if not os.path.exists(exp_path):
        os.makedirs(exp_path)

    np.save(os.path.join(exp_path, "true.npy"), y_test)
    np.save(os.path.join(exp_path, "predicted.npy"), y_predicted)

def run_all_experiments(exp, n_steps_in, n_steps_out, train_size, epochs=5, chunk_size=0):
    for n_from, n_to in NODES:
        run_experiment(f'{exp}___{n_from}to{n_to}', f'Y_{n_from}_{n_to}', n_steps_in, n_steps_out, train_size, epochs, chunk_size)

    

In [15]:
# run_all_experiments('LSTM_5_in_1_out', 5, 1, 10000, 5)
run_all_experiments('MLP_1_CHUNK_5_in_3_out', 5, 3, 10000, 10, 100)


(10000, 5)
(10000, 3)
Epoch 1/10
313/313 [==============================] - 1s 952us/step - loss: 42287.5234
Epoch 2/10
313/313 [==============================] - 0s 1ms/step - loss: 14641.3535
Epoch 3/10
313/313 [==============================] - 0s 1ms/step - loss: 14306.4980
Epoch 4/10
313/313 [==============================] - 0s 1ms/step - loss: 14032.4531
Epoch 5/10
313/313 [==============================] - 0s 1ms/step - loss: 13821.8301
Epoch 6/10
313/313 [==============================] - 0s 980us/step - loss: 13639.8350
Epoch 7/10
313/313 [==============================] - 0s 930us/step - loss: 13477.8066
Epoch 8/10
313/313 [==============================] - 0s 946us/step - loss: 13315.2988
Epoch 9/10
313/313 [==============================] - 0s 858us/step - loss: 13179.5469
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 4654.9902


10/10 [==============================] - 0s 1ms/step - loss: 30637.7695


10/10 [==============================] - 0s 1ms/step - loss: 7255.8999


10/10 [==============================] - 0s 1ms/step - loss: 16554.8242


10/10 [==============================] - 0s 2ms/step - loss: 9055.9209


...

10/10 [==============================] - 0s 2ms/step - loss: 1035.5690


10/10 [==============================] - 0s 1ms/step - loss: 607.6224


10/10 [==============================] - 0s 2ms/step - loss: 61.2105


10/10 [==============================] - 0s 2ms/step - loss: 157.0699


10/10 [==============================] - 0s 2ms/step - loss: 701.0080
